## 利用Keras預測玩家下月登入次數

### 過程
### 1. 讀取套件
### 2. 讀取資料
### 3. 使用groupby()、sum()，得出各帳號每月登入總次數
### 4. 將各月登入次數表merge在一起
### 5. 自定義 create_dataset()，轉換成時間序列分析所需的資料格式
### 6. 使用Lstm作為預測模型的演算法
### 7. 模型效能評估，使用RMSE、MAE，評估2018年05月份登入次數的預測誤差
### 8. 利用線性回歸預測帳號流失月數

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#######################################################################################
import matplotlib
import matplotlib.pyplot as plt
#matplotlib內建不支援中文，解決辦法：每次預先指定字體
matplotlib.rcParams.update({'font.size': 36})
matplotlib.use('qt4agg')
myfont = matplotlib.font_manager.FontProperties(fname='C:\\Windows\\Fonts\\msjh.ttc')
#指定字體
matplotlib.rcParams['axes.unicode_minus']=False

from matplotlib.ticker import FormatStrFormatter
majorFormatter = FormatStrFormatter('%0.f') #設定圖表浮點數的格式
%matplotlib inline 
#繪圖完就直接顯示該圖，省略每次繪圖完都要輸入plt.show指令的動作
#######################################################################################
import seaborn as sns


import numpy as np
# 設定array的float格式
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
pd.set_option('display.max_columns', 100)  #設定可顯示欄位的上限
pd.options.display.float_format = '{:,.6f}'.format  #設定浮點數的格式

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
col = ['accountid', '遊戲名稱', 'time', 'account', 'gamename_plus_time']

In [ ]:
data1 = pd.read_table('LOGIN_W2_201706_201804.txt', encoding='utf8', names=col)
data1.info()

In [ ]:
data1['登入次數'] = 1
data1.head()

In [ ]:
data1['time'] = pd.to_datetime(data1['time'])

In [ ]:
dataset = data1

dataset['year'] = dataset.time.dt.year 
dataset['month'] = dataset.time.dt.month 

dataset.head(1)

In [ ]:
dataset = dataset.drop(['遊戲名稱', 'gamename_plus_time', 'accountid'], axis=1)
dataset.head(1)

In [ ]:
dataset.info()

## 使用grouby()計算出各帳號在每月登入總次數

In [ ]:
df = dataset

df_201706 =  df[(df['year'] == 2017) & (df['month'] == 6)]
df_201707 =  df[(df['year'] == 2017) & (df['month'] == 7)]
df_201708 =  df[(df['year'] == 2017) & (df['month'] == 8)]
df_201709 =  df[(df['year'] == 2017) & (df['month'] == 9)]
df_201710 =  df[(df['year'] == 2017) & (df['month'] == 10)]
df_201711 =  df[(df['year'] == 2017) & (df['month'] == 11)]
df_201712 =  df[(df['year'] == 2017) & (df['month'] == 12)]
df_201801 =  df[(df['year'] == 2018) & (df['month'] == 1)]
df_201802 =  df[(df['year'] == 2018) & (df['month'] == 2)]
df_201803 =  df[(df['year'] == 2018) & (df['month'] == 3)]
df_201804 =  df[(df['year'] == 2018) & (df['month'] == 4)]

In [ ]:
df_201706 = df_201706.groupby(['account'])['登入次數'].sum()
df_201706 = pd.DataFrame(df_201706)
df_201706.rename(columns={'登入次數': 'num_201706'}, inplace=True)

df_201707 = df_201707.groupby(['account'])['登入次數'].sum()
df_201707 = pd.DataFrame(df_201707)
df_201707.rename(columns={'登入次數': 'num_201707'}, inplace=True)

df_201708 = df_201708.groupby(['account'])['登入次數'].sum()
df_201708 = pd.DataFrame(df_201708)
df_201708.rename(columns={'登入次數': 'num_201708'}, inplace=True)

df_201709 = df_201709.groupby(['account'])['登入次數'].sum()
df_201709 = pd.DataFrame(df_201709)
df_201709.rename(columns={'登入次數': 'num_201709'}, inplace=True)

df_201710 = df_201710.groupby(['account'])['登入次數'].sum()
df_201710 = pd.DataFrame(df_201710)
df_201710.rename(columns={'登入次數': 'num_201710'}, inplace=True)

df_201711 = df_201711.groupby(['account'])['登入次數'].sum()
df_201711 = pd.DataFrame(df_201711)
df_201711.rename(columns={'登入次數': 'num_201711'}, inplace=True)

df_201712 = df_201712.groupby(['account'])['登入次數'].sum()
df_201712 = pd.DataFrame(df_201712)
df_201712.rename(columns={'登入次數': 'num_201712'}, inplace=True)

df_201801 = df_201801.groupby(['account'])['登入次數'].sum()
df_201801 = pd.DataFrame(df_201801)
df_201801.rename(columns={'登入次數': 'num_201801'}, inplace=True)

df_201802 = df_201802.groupby(['account'])['登入次數'].sum()
df_201802 = pd.DataFrame(df_201802)
df_201802.rename(columns={'登入次數': 'num_201802'}, inplace=True)

df_201803 = df_201803.groupby(['account'])['登入次數'].sum()
df_201803 = pd.DataFrame(df_201803)
df_201803.rename(columns={'登入次數': 'num_201803'}, inplace=True)

df_201804 = df_201804.groupby(['account'])['登入次數'].sum()
df_201804 = pd.DataFrame(df_201804)
df_201804.rename(columns={'登入次數': 'num_201804'}, inplace=True)

In [ ]:
data2 = pd.read_table('LOGIN_W2_201712_201805.txt', encoding='utf8', names=col)
data2.info()

In [ ]:
data2['登入次數'] = 1
data2['time'] = pd.to_datetime(data2['time'])
data2.head()

In [ ]:
dataset = data2

dataset['year'] = dataset.time.dt.year 
dataset['month'] = dataset.time.dt.month 
dataset = dataset.drop(['遊戲名稱', 'gamename_plus_time', 'accountid'], axis=1)

dataset.head(1)

In [ ]:
dataset.info()

In [ ]:
df_201805 = dataset[(dataset['year'] == 2018) & (dataset['month'] == 5)]

In [ ]:
df_201805 = df_201805.groupby(['account'])['登入次數'].sum()
df_201805 = pd.DataFrame(df_201805)
df_201805.rename(columns={'登入次數': 'num_201805'}, inplace=True)

## 帳號的每月登入次數表

In [ ]:
dataframe = [df_201708,df_201709,df_201710,df_201711, df_201712, df_201801, df_201802,df_201803, df_201804, df_201805]

In [ ]:
full_data = pd.merge(df_201706, df_201707, left_index=True, right_index=True, how='outer')
for i in dataframe:
    full_data = pd.merge(full_data, i, left_index=True, right_index=True, how='outer')

In [ ]:
full_data = full_data[~full_data.index.duplicated(keep='last')]
full_data.index.shape

In [ ]:
full_data = full_data.replace(np.nan, 0, regex=True)
full_data.head()

In [ ]:
full_data.describe()

In [ ]:
full_data.info()

In [ ]:
X = full_data.iloc[2,:-1]
X

In [ ]:
plt.plot(X)
plt.xticks(rotation=90)

## 創建訓練樣本
### 使用201706~201804月份的數據

### 訓練時使用201706~201803月份去預測201804月份
### 根據前六個月的資料預測下一月份

In [ ]:
train = full_data.iloc[:, :-1] #排除201805
train.head(1)

In [ ]:

def create_dataset(look_back=1):
    dataX, dataY = [], []
    for i in range(len(train.index)):
        X = train.iloc[i, :].values 
        X = X.reshape(-1,1)
        for i in range(len(X) - look_back):
            a = X[i:(i + look_back)]
            dataX.append(a)
            dataY.append(X[i + look_back])
    return np.array(dataX), np.array(dataY)


In [ ]:

lookback = 3
train_x, train_y = create_dataset(lookback)
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))

print(train_x.shape, train_y.shape)


In [ ]:

model = Sequential()
model.add(LSTM(32,input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))
model.add(LSTM(16))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
history = model.fit(train_x, train_y, epochs=1, batch_size=64, verbose=1, shuffle=False)


In [ ]:

plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='best')


## LSTM模型預測 5月份登入次數

In [ ]:

%%time
prediction = []

for i in range(len(train.index)):
    data = train.iloc[i,:] 
    topred = data[-lookback:] 
    topred = topred.reshape(1,topred.shape[0], 1)
    yhat = model.predict(topred)
    yhat = yhat.reshape(-1)
    yhat = yhat.tolist()
    
    prediction.append(yhat[0])


In [ ]:
prediction = np.array(prediction)

## 模型評估

In [ ]:
y_true = full_data['num_201805'].values

In [ ]:
# RMSE
score = np.sqrt(mean_squared_error(y_true, prediction))
score

In [ ]:
# MAE
score = mean_absolute_error(y_true, np.round(prediction))
score

In [ ]:
score = np.sqrt(mean_squared_error(y_true, np.round(prediction)))
score

## Sklearn線性回歸
### 繪製各帳號的回歸線，並依斜率判斷帳號登入次數趨勢是上升還是下降

In [ ]:
def cre_dataset(x):
    dataX, dataY = [], []
    for i in range(len(x)):
        a = i
        dataX.append(a)
        dataY.append(x[i])
    return np.array(dataX), np.array(dataY)

In [ ]:
model01 = LinearRegression()

In [ ]:

%%time

coefficient = []

for i in range(len(full_data.index)):
    data = full_data.iloc[i,:]
    train_x, train_y = cre_dataset(data)
    train_x = train_x.reshape(-1,1)
    
    model01.fit(train_x, train_y)
    coef = model01.coef_
    coefficient.append(coef)


In [ ]:
dd = full_data.iloc[2,:]
train_x, train_y = cre_dataset(dd)
train_x = train_x.reshape(-1,1)

model01.fit(train_x, train_y)
trend = model01.predict(train_x)

print('coef:', model01.coef_[0])
plt.plot(train_y)
plt.plot(trend)
plt.title('歷史登入次數', fontproperties=myfont, fontsize=20)
plt.xlabel('時間(單位:月)', fontproperties=myfont, fontsize=16)
plt.ylabel('登入次數', fontproperties=myfont, fontsize=16)

plt.legend(['true', 'prediction'], loc='best')

In [ ]:
#挑出登入趨勢下降的帳號
churn_dataset = full_data.copy()
churn_dataset['coef'] = coefficient
churn_dataset = churn_dataset[churn_dataset['coef'] < 0]

In [ ]:
churn_dataset.info()

## 依回歸線與X軸的交點推斷出帳號流失的月份

In [ ]:
%%time

x_test = np.array([13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30])
x_test = x_test.reshape(-1,1)

churn_month = []
for i in range(len(churn_dataset.index)):
    data = churn_dataset.iloc[i,:]
    train_x, train_y = cre_dataset(data)
    train_x = train_x.reshape(-1,1)
    quadratic_featurizer = PolynomialFeatures(degree=2)

    X_train = quadratic_featurizer.fit_transform(train_x)

    regressor = LinearRegression()
    regressor.fit(X_train, train_y)
    
    X_test = quadratic_featurizer.fit_transform(x_test)
    trend = regressor.predict(X_test)
    
    month = 0
    for j in range(len(trend)):
        if trend[j] > 0:
            month += 1
        elif trend[j] <= 0:
            churn_month.append(month)
            break
        

In [ ]:
len(churn_month)

In [ ]:
dd = churn_dataset.iloc[2,:]
train_x, train_y = cre_dataset(dd)
train_x = train_x.reshape(-1,1)

quadratic_featurizer = PolynomialFeatures(degree=2)

X_train = quadratic_featurizer.fit_transform(train_x)
regressor = LinearRegression()
regressor.fit(X_train, train_y)
X_test = quadratic_featurizer.fit_transform(x_test)
trend = regressor.predict(X_train)

plt.plot(train_y)
plt.plot(trend)
plt.title('歷史登入次數', fontproperties=myfont, fontsize=20)
plt.xlabel('時間(單位:月)', fontproperties=myfont, fontsize=16)
plt.ylabel('登入次數', fontproperties=myfont, fontsize=16)


In [ ]:
churn_dataset['流失月數預測'] = churn_month
churn_dataset = churn_dataset[['流失月數預測']]

In [ ]:
churn_dataset.head()

In [ ]:
full_data = pd.merge(full_data, churn_dataset,left_index=True, right_index=True, how='outer')
full_data = full_data.fillna(full_data.流失月數預測.max()) #遺失值用最大值填補
full_data.head(2)

## 預測2018年06月份登入次數

In [ ]:

%%time
pred_201806 = []

for i in range(len(full_data.index)):
    data = full_data.values
    topred = data[i, -lookback:]
    topred = topred.reshape(1,topred.shape[0], 1)
    yhat = model.predict(topred)
    yhat = yhat.reshape(-1)
    yhat = yhat.tolist()
    
    pred_201806.append(yhat[0])


In [ ]:
result = pd.DataFrame({'登入趨勢': coefficient,
                       '下月登入次數預測': np.round(pred_201806),
                       '遊戲帳號': full_data.index.values,
                       '流失月數預測': full_data.流失月數預測.values})

In [ ]:
result.loc[(result['下月登入次數預測'] < 0), '下月登入次數預測']= 0

In [ ]:
# 3代表上升 2代表持平 1代表下降
result.loc[(result['登入趨勢'] > 0), '登入趨勢']= 3
result.loc[(result['登入趨勢'] == 0), '登入趨勢']= 2
result.loc[(result['登入趨勢'] < 0), '登入趨勢']= 1

In [ ]:
full_data.head()

In [ ]:
result.head()

In [ ]:
#輸出成csv檔
#result.to_csv('pred.csv', encoding='utf_8', index=False)